In [2]:
!pip install fastai --upgrade -q
from fastai.vision.all import *
import warnings
warnings.filterwarnings("ignore")

!pip install wwf -q
!pip install timm -q
from wwf.vision.timm import *

!pip install efficientnet_pytorch -q

In [7]:


train_df = pd.read_csv('train_v2.csv')
print (train_df)
def get_x(r):
    return path/'train-jpg'/(r['image_name']+'.jpg')

def get_y(r):
    return r['tags'].split()

def get_data(size=224,bs=64,data_df=train_df):
    dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                       splitter=RandomSplitter(seed=42),
                       get_x=get_x,
                       get_y=get_y,
                       item_tfms = Resize(size),
                       batch_tfms = [*aug_transforms(flip_vert=True,max_warp=0),
                                     Normalize.from_stats(*imagenet_stats)]
                      )
    return dblock.dataloaders(data_df,bs=bs)

        image_name                                           tags
0          train_0                                   haze primary
1          train_1                agriculture clear primary water
2          train_2                                  clear primary
3          train_3                                  clear primary
4          train_4      agriculture clear habitation primary road
...            ...                                            ...
40474  train_40474                                  clear primary
40475  train_40475                                         cloudy
40476  train_40476                      agriculture clear primary
40477  train_40477                 agriculture clear primary road
40478  train_40478  agriculture cultivation partly_cloudy primary

[40479 rows x 2 columns]


In [ ]:
dls = get_data(300,40)
dls.show_batch(nrows=1, ncols=3)

In [ ]:

# f2macro = FBetaMulti(beta=2,average='macro')
# f2micro = FBetaMulti(beta=2,average='micro')
f2samples = FBetaMulti(beta=2,average='samples',thresh=0.2)

# model = EfficientNet.from_pretrained('efficientnet-b7', get_c(dls))
metrics = [partial(accuracy_multi, thresh=0.2), f2samples]
cbs = [MixUp]
# learn = cnn_learner(dls, resnet50, metrics=metrics, cbs=cbs)
# learn = Learner(dls, model, metrics=metrics, f2samples], cbs=cbs)
# learn = Learner(dls, model, metrics=metrics, loss_func=LabelSmoothingCrossEntropy(), cbs=cbs)

learn = timm_learner(dls, 'efficientnet_b3', metrics=metrics, cbs=cbs)

# learn.lr_find()
learn.fine_tune(12, base_lr=3e-2, freeze_epochs=4)